## Задача 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
import pandas as pd

In [75]:
df = pd.read_csv('ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [19]:
def class_(param):
    
    if param <= 2:
        return 'низкий рейтинг'
    elif param > 2 and param <= 4:
        return 'средний рейтинг'
    elif param >= 4.5 and param <= 5:
        return 'высокий рейтинг'

In [22]:
df['class'] = df['rating'].apply(class_)
df.head(4)

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг


## Задача 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [74]:
stats = pd.read_csv('keywords.csv')
stats.head(1)

,keyword,shows
0,вк,64292779


In [60]:
def sity(param):
    search = 0
    geo_data = { 'Центр': ['москва', 'тула', 'ярославль'], 'Северо-Запад': ['петербург', 'псков', 'мурманск'], 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}
    
    for word in param.split(' '):
        for k, v in geo_data.items():
            if word in v:
                return k
                search = 1
    if search == 0:
        return 'undefined'

In [73]:
stats['region'] = stats['keyword'].apply(sity)
stats.sort_values('region', ascending = False).head()

,keyword,shows,region
79598,метро схема москва,62915,Центр
94442,скорый москва россия,3866,Центр
46003,гей знакомства москва,8436,Центр
81512,локомотив москва,4395,Центр
71891,москва слезам не верит смотреть бесплатно в хо...,5167,Центр


## Задача 3 (бонусная)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга


In [248]:
import numpy as np
import re

In [249]:
film = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [250]:
def production_year(param):
    import re
    search = 0
    years = np.arange(1950, 2011)
    
    list_ = param.split(' ')
    
    for el in list_:
        el_clear = re.sub(r'[\(\)]','', el)
       
        if el_clear.isdigit():
        
            if int(el_clear) in years:
                return el_clear
                search = 1
    
    
    if search == 0:
        return 1900

In [276]:
film['year'] = film['title'].apply(production_year)

In [277]:
film_rating = film.merge(ratings, how='outer')

In [286]:
film_rating.groupby('year').mean().sort_values('rating', ascending = False).filter(items = ['year', 'movieId', 'rating']).head(20)

,movieId,rating
year,,
1957,2814.920530,4.067797
1952,2512.939850,4.061538
1954,2508.187879,4.044343
1972,3727.883991,4.029621
1974,3106.331384,4.023622
1951,2005.373444,3.991489
1950,3572.729258,3.977679
1962,4070.741047,3.957865
1977,4377.205052,3.911720
